In [32]:
import pandas as pd
import regex as re

In [33]:
data = pd.read_csv('log_week_end.csv', names=['rn', 'loguser', 'q'])

C:\Users\darkw\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [34]:
data.sample(5)

,rn,loguser,q
61203,262117,etl_1151,"into tbl_9323,join tbl_9323"
577588,766823,etl_1151,"join tbl_15844,join tbl_15850,join tbl_27892,j..."
306083,79314,dev_332,"from tbl_7207,from tbl_11022"
600485,127394,etl_2048,from tbl_99699
576735,822040,etl_1151,from tbl_18799


Собираем в списки все команды

In [35]:
intos, froms, joins = [], [], []

In [36]:
for x in data.q.str.split(','):
    for g in x:
        if 'from' in g:
            froms.append(g)
        if 'join' in g:
            joins.append(g)
        if 'into' in g:
            intos.append(g)

Оставляем только название таблиц

In [37]:
intos_tbs = [re.search('tbl_\d*', x).group(0) for x in intos]
froms_tbs = [re.search('tbl_\d*', x).group(0) for x in froms]
joins_tbs = [re.search('tbl_\d*', x).group(0) for x in joins]

Делаем датафрейм с окличество into и from (from только от пользователей dev)

In [38]:
df1 = pd.DataFrame(data = froms_tbs, columns=['col']).groupby('col').agg({'col': 'count'})
df1.columns= ['count_froms']
df1.reset_index(inplace=True)

In [39]:
df1

,col,count_froms
0,tbl_10000,24
1,tbl_10001,24
2,tbl_100026,6
3,tbl_100027,6
4,tbl_100028,6
...,...,...
34774,tbl_9996,24
34775,tbl_9997,24
34776,tbl_9998,24
34777,tbl_99986,20


In [40]:
df2 = pd.DataFrame(data = intos_tbs, columns=['col']).groupby('col').agg({'col': 'count'})
df2.columns= ['count_intos']
df2.reset_index(inplace=True)
df2

,col,count_intos
0,tbl_10000,12
1,tbl_10001,12
2,tbl_100026,6
3,tbl_100027,6
4,tbl_100028,6
...,...,...
25240,tbl_9995,12
25241,tbl_9996,12
25242,tbl_9997,12
25243,tbl_9998,12


In [41]:
df = df1.merge(df2, on ='col', how='outer')

In [42]:
df

,col,count_froms,count_intos
0,tbl_10000,24.0,12.0
1,tbl_10001,24.0,12.0
2,tbl_100026,6.0,6.0
3,tbl_100027,6.0,6.0
4,tbl_100028,6.0,6.0
...,...,...,...
41267,tbl_97783,NaN,21.0
41268,tbl_97784,NaN,21.0
41269,tbl_99127,NaN,6.0
41270,tbl_99134,NaN,6.0
